In [1]:
import time
import datetime
from mkndaq.inst.neph import NEPH

cfg = {'ne300': {
                'type': 'NE300',
                'serial_number': '23-0690',
                'serial_id': 0,
                'protocol': 'acoem',
                'socket': {
                    'host': '192.168.0.50',
                    'port': 32783,
                    'timeout': 5,
                },
                # 'set_datetime': '',
                # 'sampling_interval': '1',
                'staging_zip': True,  
                'verbosity': 2,  # 0: silent, 1: medium, 2: full          
            },
                'reporting_interval': '10',
                'logs': 'mkndaq/logs',
                'data': 'mkndaq/data',
                'staging': {
                    'path': 'mkndaq/staging',
                },
}

ne300 = NEPH('ne300', cfg)

# legacy
# # Initialize NEPH (name: ne300  S/N: 23-0690)
#   Instrument identified itself as '{'id': 'Aurora NE-300 v5.0 ID #230690'}'.
# acoem
# # Initialize NEPH (name: ne300  S/N: 23-0690)
#   Instrument identified itself as '{'Model': 158, 'Variant': 300, 'Sub-Type': 0, 'Range': 0, 'Build': 158, 'Branch': 300}'.

# Initialize NEPH (name: ne300  S/N: 23-0690)


  Instrument identified itself as '{'Model': 158, 'Variant': 300, 'Sub-Type': 0, 'Range': 0, 'Build': 158, 'Branch': 300}'.


In [2]:
res = ne300.get_id()
print(res)
# legacy
# {'id': 'Aurora NE-300 v5.0 ID #230690'}
# acoem
# {'Model': 158, 'Variant': 300, 'Sub-Type': 0, 'Range': 0, 'Build': 158, 'Branch': 300}

{'Model': 158, 'Variant': 300, 'Sub-Type': 0, 'Range': 0, 'Build': 158, 'Branch': 300}


In [3]:
ne300.get_current_data(sep="|")
# legacy
# '8/5/2024 12:26:39|0.173713|-0.347197|0.776668|0.191542|0.839787|0.231125|27.45|0|0.00|976.0|0|0'
# acoem
# Not implemented.

Not implemented.


''

In [ ]:
parameter_map = {'legacy': 'acoem',
                 99: [[80, 81],1, 1635000,1525000,1450000,1635090,1525090,1450090,5001,5004,5003,5002,4036,4035],
                 }
parameter_map_legacy = {99: [[80, 81], 1, 30, 2, 31, 3, 32, 18, [], [], 19, 90]}

parameters = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,16,18,19] #15,17,
d12 = ne300.get_values(parameters=parameters)
print(d12)
parameters = [20,21,22,26,27,28,30,31,32,33,35,36,37,38,] #23,24,25,29,34,39
d23 = ne300.get_values(parameters=parameters)
print(d23)
parameters = [40,41,] #42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59]
d45 = ne300.get_values(parameters=parameters)
print(d45)
parameters = [66,68,69,71,] #60,61,62,63,64,65,67,70,72,73,74,75,76,77,78,79
d67 = ne300.get_values(parameters=parameters)
print(d67)
parameters = [80,81,82,83,84,90,98,99] #85,86,87,88,89,91,92,93,94,95,96,97,
d89 = ne300.get_values(parameters=parameters)
print(d89)
# legacy
# {0: '0.03', 1: '0.123338', 2: '0.865954', 3: '0.953804', 4: '253.337814', 5: '253.337814', 6: '1592424.875000', 7: '14685.080078', 8: '0.009768', 9: '1968038.375000', 10: '19835.837891', 11: '0.010278', 12: '1452458.250000', 13: '19762.300781', 14: '0.013365', 16: '5.02', 18: '27.23', 19: '975.9'}
# {20: '33.52', 21: '29.61', 22: '971.7', 26: '635', 27: '525', 28: '450', 30: '-0.324081', 31: '0.244772', 32: '0.285640', 33: '11512.603516', 35: '13811.673828', 36: '0.007453', 37: '13133.730469', 38: '0.007316'}
# {40: '0.008941', 41: '0.003621'}
# {66: 'mb', 68: 'Monitr', 69: 'Normal', 71: '000'}
# {80: '8/5/2024', 81: '12:14:10', 82: '0.005000', 83: '0.008313', 84: '0.006109', 90: '0x0', 98: '2,0,90', 99: '8/5/2024 12:14:10, 0.123338, -0.324081, 0.865954, 0.244772, 0.953804, 0.285640, 27.23,0, 0.00, 975.9,0,0'}
# acoem


In [4]:
data = ne300.get_new_data()
print(f"len: {len(data)}")
print(data)

# legacy
# len: 1118
# 30/11/2023 00:00:00,60,0,9.765964e-01,1.155139e+00,1.231401e+00,1.073153e-01,1.748719e-01,2.522535e-01,3.354000e+01,1.002410e+03,2.996000e+01,2.644000e+01,1.004090e+03,5.332000e+01,6.020748e+00,5.000000e-03,5.000000e-03,5.000000e-03,2.933394e+02,1.494557e+06,1.888788e+06,1.392557e+06,1.470679e+04,1.969712e+04,1.927430e+04,1.087828e+04,1.395863e+04,1.322223e+04,2.410400e+01,1.879666e+03,1.839111e+01,5.090332e+00,2.026616e+01,1.035651e-02,1.090555e-02,1.380057e-02,7.627005e-03,7.697072e-03,9.421869e-03,7.000000e+01,2.900000e+01,4.880183e+01,0.000000e+00,
# 30/11/2023 00:01:00,60,0,9.642181e-01,1.143413e+00,1.238636e+00,1.114373e-01,1.734347e-01,2.362185e-01,3.351000e+01,1.002670e+03,2.996000e+01,2.638000e+01,1.003850e+03,5.332000e+01,5.982811e+00,1.746927e-02,5.000000e-03,5.000000e-03,2.433375e+02,1.493905e+06,1.891611e+06,1.389815e+06,1.404880e+04,1.947819e+04,1.927932e+04,1.091333e+04,1.383672e+04,1.295173e+04,2.412598e+01,1.708967e+03,1.841309e+01,4.965820e+00,2.026616e+01,9.923033e-03,1.079363e-02,1.386724e-02,7.669333e-03,7.611208e-03,9.236093e-03,7.000000e+01,3.000000e+01,4.895999e+01,0.000000e+00,
# acoem


Not implemented.
len: 0



In [5]:
data = ne300.get_all_data()
print(f"len: {len(data)}")
print(data)

# legacy
# len: 1118
#
# 30/11/2023 00:00:00,60,0,9.765964e-01,1.155139e+00,1.231401e+00,1.073153e-01,1.748719e-01,2.522535e-01,3.354000e+01,1.002410e+03,2.996000e+01,2.644000e+01,1.004090e+03,5.332000e+01,6.020748e+00,5.000000e-03,5.000000e-03,5.000000e-03,2.933394e+02,1.494557e+06,1.888788e+06,1.392557e+06,1.470679e+04,1.969712e+04,1.927430e+04,1.087828e+04,1.395863e+04,1.322223e+04,2.410400e+01,1.879666e+03,1.839111e+01,5.090332e+00,2.026616e+01,1.035651e-02,1.090555e-02,1.380057e-02,7.627005e-03,7.697072e-03,9.421869e-03,7.000000e+01,2.900000e+01,4.880183e+01,0.000000e+00,
# 30/11/2023 00:01:00,60,0,9.642181e-01,1.143413e+00,1.238636e+00,1.114373e-01,1.734347e-01,2.362185e-01,3.351000e+01,1.002670e+03,2.996000e+01,2.638000e+01,1.003850e+03,5.332000e+01,5.982811e+00,1.746927e-02,5.000000e-03,5.000000e-03,2.433375e+02,1.493905e+06,1.891611e+06,1.389815e+06,1.404880e+04,1.947819e+04,1.927932e+04,1.091333e+04,1.383672e+04,1.295173e+04,2.412598e+01,1.708967e+03,1.841309e+01,4.965820e+00,2.026616e+01,9.923033e-03,1.079363e-02,1.386724e-02,7.669333e-03,7.611208e-03,9.236093e-03,7.000000e+01,3.000000e+01,4.895999e+01,0.000000e+00,

# acoem


Not implemented.
len: 0



In [6]:
ne300.get_instr_type()

# acoem
# [158, 300, 0, 0]

[158, 300, 0, 0]

In [3]:
ne300.get_datetime()
# legacy
# datetime.datetime(2024, 5, 8, 12, 31, 24)
# acoem
# datetime.datetime(2024, 5, 8, 12, 38, 28)

datetime.datetime(2024, 5, 8, 14, 48, 14)

In [8]:
logging_config = ne300.get_logging_config()
print(logging_config)
# acoem
# [41, 2635000, 2525000, 2450000, 2635090, 2525090, 2450090, 5001, 5002, 5003, 5004, 5005, 5006, 5010, 26635000, 26525000, 26450000, 13525000, 15635000, 15525000, 15450000, 11635000, 11525000, 11450000, 11635090, 11525090, 11450090, 6007, 6008, 6001, 6002, 6003, 6635000, 6525000, 6450000, 6635090, 6525090, 6450090, 5011, 5012, 5014, 4035]

[41, 2635000, 2525000, 2450000, 2635090, 2525090, 2450090, 5001, 5002, 5003, 5004, 5005, 5006, 5010, 26635000, 26525000, 26450000, 13525000, 15635000, 15525000, 15450000, 11635000, 11525000, 11450000, 11635090, 11525090, 11450090, 6007, 6008, 6001, 6002, 6003, 6635000, 6525000, 6450000, 6635090, 6525090, 6450090, 5011, 5012, 5014, 4035]


In [4]:
# cycle through operating states
# 0: ambient, 1: zero, 2: span
for i in range(3):
    print(f"current: {ne300.get_current_operation()} > set: {i} ...")
    print(f"done: {ne300.set_current_operation(state=i)} okay")
    time.sleep(3)
print(f"current: {ne300.get_current_operation()} > set: {0} ...")
print(f"done: {ne300.set_current_operation(state=0)} okay")

# acoem
# current: 0 > set: 0 ...
# timed out
# done: 0 okay
# current: 0 > set: 1 ...
# timed out
# done: 1 okay
# current: 1 > set: 2 ...
# timed out
# done: 2 okay
# current: 2 > set: 0 ...
# timed out
# done: 0 okay

current: 0 > set: 0 ...


timed out
done: 0 okay
current: 0 > set: 1 ...
timed out
done: 1 okay
current: 1 > set: 2 ...
timed out
done: 2 okay
current: 2 > set: 0 ...


In [14]:
from datetime import datetime, timezone, timedelta
end = datetime.now(timezone.utc)
start = end - timedelta(minutes=5)
print(f"start: {start}\nend  : {end}")
data = ne300.get_logged_data(start=start, end=end, verbosity=0)
for i in range(len(data)):
    print(data[i])

start: 2024-05-08 14:40:35.938689+00:00
end  : 2024-05-08 14:45:35.938689+00:00
{}
{}
{2635000: 0.19195599853992462, 2525000: 0.8410816192626953, 2450000: 0.8065739274024963, 2635090: -0.444142609834671, 2525090: 0.24221180379390717, 2450090: 0.18188859522342682, 5001: 28.240020751953125, 5002: 974.4099731445312, 5003: 5.119999885559082, 5004: 29.770009994506836, 5005: 971.8400268554688, 5006: 33.27000045776367, 5010: 0.0, 26635000: 0.006195979192852974, 26525000: 0.007089253049343824, 26450000: 0.004999999888241291, 13525000: 366.67608642578125, 15635000: 1587764.0, 15525000: 1967904.0, 15450000: 1451798.0, 11635000: 14793.6396484375, 11525000: 19870.939453125, 11450000: 19461.48046875, 11635090: 10859.919921875, 11525090: 13778.2802734375, 11450090: 13100.330078125, 6007: 24.221189498901367, 6008: 1689.1190185546875, 6001: 18.413089752197266, 6002: 4.91455078125, 6003: 21.465429306030273, 6635000: 0.009652410633862019, 6525000: 0.010425649583339691, 6450000: 0.013291900046169758, 663

In [13]:
ne300.get_values(parameters=[4035])

{4035: 0}

In [ ]:
# print(ne300.do_zero_check())
# print(ne300.get_all_new_data(save=False, sep="|"))

# Aurora NE-300 v5.0 ID #230690

# (0, 'Aurora NE-300 v5.0 ID #230690\r\n')
# 2024-01-25 14:37:37 .get_data (name=ne300, save=False)
# (0, '28/2/2024 15:57:00|60|0|1.341209e+01|1.900581e+01|2.459902e+01|2.218892e+00|2.862914e+00|3.427358e+00|2.630001e+01|9.678900e+02|2.457000e+01|2.895999e+01|9.690800e+02|2.708000e+01|6.007494e+00|1.470621e-02|1.693243e-02|1.907027e-02|2.333371e+02|1.522294e+06|1.909421e+06|1.402254e+06|1.789572e+04|2.621301e+04|2.658940e+04|1.161944e+04|1.474019e+04|1.391521e+04|2.418457e+01|1.744695e+03|1.836182e+01|4.848633e+00|2.049459e+01|1.227465e-02|1.426282e-02|1.897962e-02|7.924382e-03|7.973331e-03|9.864524e-03|6.607600e+01|0.000000e+00|5.020050e+01|0.000000e+00|\r\n\n28/2/2024 15:57:00|60|0|1.341209e+01|1.900581e+01|2.459902e+01|2.218892e+00|2.862914e+00|3.427358e+00|2.630001e+01|9.678900e+02|2.457000e+01|2.895999e+01|9.690800e+02|2.708000e+01|6.007494e+00|1.470621e-02|1.693243e-02|1.907027e-02|2.333371e+02|1.522294e+06|1.909421e+06|1.402254e+06|1.789572e+04|2.621301e+04|2.658940e+04|1.161944e+04|1.474019e+04|1.391521e+04|2.418457e+01|1.744695e+03|1.836182e+01|4.848633e+00|2.049459e+01|1.227465e-02|1.426282e-02|1.')
# Instrument ne300 status: 30/11/2023 00:00:00,60,0,9.765964e-01,1.155139e+00,1.231401e+00,1.073153e-01,1.748719e-01,2.522535e-01,3.354000e+01,1.002410e+03,2.996000e+01,2.644000e+01,1.004090e+03,5.332000e+01,6.020748e+00,5.000000e-03,5.000000e-03,5.000000e-03,2.933394e+02,1.494557e+06,1.888788e+06,1.392557e+06,1.470679e+04,1.969712e+04,1.927430e+04,1.087828e+04,1.395863e+04,1.322223e+04,2.410400e+01,1.879666e+03,1.839111e+01,5.090332e+00,2.026616e+01,1.035651e-02,1.090555e-02,1.380057e-02,7.627005e-03,7.697072e-03,9.421869e-03,7.000000e+01,2.900000e+01,4.880183e+01,0.000000e+00,
# 30/11/2023 00:01:00,60,0,9.642181e-01,1.143413e+00,1.238636e+00,1.114373e-01,1.734347e-01,2.362185e-01,3.351000e+01,1.002670e+03,2.996000e+01,2.638000e+01,1.003850e+03,5.332000e+01,5.982811e+00,1.746927e-02,5.000000e-03,5.000000e-03,2.433375e+02,1.493905e+06,1.891611e+06,1.389815e+06,1.404880e+04,1.947819e+04,1.927932e+04,1.091333e+04,1.383672e+04,1.295173e+04,2.412598e+01,1.708967e+03,1.841309e+01,4.965820e+00,2.026616e+01,9.923033e-03,1.079363e-02,1.
# (0, '30/11/2023 00:00:00,60,0,9.765964e-01,1.155139e+00,1.231401e+00,1.073153e-01,1.748719e-01,2.522535e-01,3.354000e+01,1.002410e+03,2.996000e+01,2.644000e+01,1.004090e+03,5.332000e+01,6.020748e+00,5.000000e-03,5.000000e-03,5.000000e-03,2.933394e+02,1.494557e+06,1.888788e+06,1.392557e+06,1.470679e+04,1.969712e+04,1.927430e+04,1.087828e+04,1.395863e+04,1.322223e+04,2.410400e+01,1.879666e+03,1.839111e+01,5.090332e+00,2.026616e+01,1.035651e-02,1.090555e-02,1.380057e-02,7.627005e-03,7.697072e-03,9.421869e-03,7.000000e+01,2.900000e+01,4.880183e+01,0.000000e+00,\r\n30/11/2023 00:01:00,60,0,9.642181e-01,1.143413e+00,1.238636e+00,1.114373e-01,1.734347e-01,2.362185e-01,3.351000e+01,1.002670e+03,2.996000e+01,2.638000e+01,1.003850e+03,5.332000e+01,5.982811e+00,1.746927e-02,5.000000e-03,5.000000e-03,2.433375e+02,1.493905e+06,1.891611e+06,1.389815e+06,1.404880e+04,1.947819e+04,1.927932e+04,1.091333e+04,1.383672e+04,1.295173e+04,2.412598e+01,1.708967e+03,1.841309e+01,4.965820e+00,2.026616e+01,9.923033e-03,1.079363e-02,1')
# 2024-01-25 14:23:26 .get_data (name=ne300, save=False)
# (0, '28/2/2024 15:43:17,13.810564,2.414614,19.843826,2.683266,24.489584,3.266213,25.60,0,0.00,967.7,0,0\r\n')
# 2024-01-25 14:23:36 .get_data (name=ne300, save=False)
# (0, '28/2/2024 15:43:28,13.763916,2.429190,19.711246,2.699752,24.422705,3.274734,25.61,0,0.00,967.8,0,0\r\n')
# 2024-01-25 14:23:47 .get_data (name=ne300, save=False)
# (0, '28/2/2024 15:43:38,13.717136,2.425864,19.598263,2.699717,24.439505,3.250718,25.61,0,0.00,967.7,0,0\r\n')
# # Initialize NE300 (name: ne300  S/N: 23-0690)
# b'\x020x0'

In [ ]:
# import datetime
# import polars as pl
# import struct
# response = b'\x02\x00\x07\x03\x07\xe8\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00*\x00(4\xf8\x00&\x87H\x00%bP\x00(5R\x00&\x87\xa2\x00%b\xaa\x00\x00\x13\x89\x00\x00\x13\x8a\x00\x00\x13\x8b\x00\x00\x13\x8c\x00\x00\x13\x8d\x00\x00\x13\x8e\x00\x00\x13\x92\x01\x96j\xf8\x01\x94\xbdH\x01\x93\x98P\x00\xce`\x08\x00\xee\x928\x00\xec\xe4\x88\x00\xeb\xbf\x90\x00\xb1\x898\x00\xaf\xdb\x88\x00\xae\xb6\x90\x00\xb1\x89\x92\x00\xaf\xdb\xe2\x00\xae\xb6\xea\x00\x00\x17w\x00\x00\x17x\x00\x00\x17q\x00\x00\x17r\x00\x00\x17s\x00e=\xf8\x00c\x90H\x00bkP\x00e>R\x00c\x90\xa2\x00bk\xaa\x00\x00\x13\x93\x00\x00\x13\x94\x00\x00\x13\x96\x00\x00\x0f\xc3\x00\x00\x00\x00\x00\x02\x00\x00a<\xf1@\x00\x00\x00<\x00\x00\x00*>M\xe7\xc2?1\xcb`?\x1f\x00A\xbe\x9f\xd3\xe2=\xff\x1c\xc6=\xe0\xce\x84A\xe5\xc2\x8fDo\xfdq?\xfe\xb8RA\xf3\xd7\x0fDpQHA\xd1\xd7\n;\x1c'
# verbosity=2

# def __acoem_timestamp_to_datetime(timestamp: int) -> datetime.datetime:
#     try:
#         dtm = timestamp
#         SS = dtm % 64
#         dtm = dtm // 64
#         MM = dtm % 64
#         dtm = dtm // 64
#         HH = dtm % 32
#         dtm = dtm // 32
#         dd = dtm % 32
#         dtm = dtm // 32
#         mm = dtm % 16
#         yyyy = dtm // 16 + 2000

#         return datetime.datetime(yyyy, mm, dd, HH, MM, SS)

#     except Exception as err:
#         print(err)
#         return datetime.datetime(1111, 1, 1, 1, 1, 1)

# def __acoem_decode_logged_data(response: bytes, verbosity: int=0) -> dict():
#     if response[2] == 7:
#         # get_logger_data command (7) sent
#         message_length = int(int.from_bytes(response[4:6]) / 4)
#         response_body = response[6:-2]
#         fields_per_record = int.from_bytes(response_body[12:16])
#         items_per_record = fields_per_record + 4
#         number_of_records = message_length // items_per_record
#         if verbosity>1:
#             print(f"message length (items): {message_length}")
#             print(f"response body length  : {len(response_body)}")
#             print(f"response body (bytes) : {response_body}")
#             print(f"number of records     : {number_of_records}")

#         # parse bytearray into records and records into dict of header record(s) and data records
#         records = [response_body[(i*items_per_record*4):((i+1)*(items_per_record*4)-1)] for i in range(number_of_records)]
#         keys = []
#         values = []
#         data = dict()
#         # for i in range(number_of_records):
#         for i in range(2):
#             if records[i][0]==1:
#                 # header record
#                 number_of_fields = int.from_bytes(records[i][12:16])
#                 keys = [int.from_bytes(records[i][(16 +j*4):(16 + (j+1)*4)]) for j in range(number_of_fields)]
#             if records[i][0]==0:
#                 # data record
#                 number_of_fields = int.from_bytes(records[i][12:16])
#                 values = [records[i][(16 +j*4):(16 + (j+1)*4)] for j in range(number_of_fields)]

#                 data = dict(zip(keys, values))
#                 for k, v in data.items():
#                     data[k] = struct.unpack('>f', v)[0] if (k>1000 and len(v)>0) else None
#                 data['dtm'] = __acoem_timestamp_to_datetime(int.from_bytes(records[i][4:8]))
#                 #  1631383872 b'a<\xf1@'
#                 data['logging_interval'] = int.from_bytes(records[i][8:12])
#                 # item  48 (bytes  192- 195): 60 None b'\x00\x00\x00<'
#             if verbosity>1:
#                 print(f"record  {i:2.0f}: {records[i]}")
#                 print(f"type    : {records[i][0]}")
#                 print(f"inst op : {records[i][0]}")
#                 print(f"{i}: keys: {keys}")
#                 print(f"{i}: values: {values}")
#                 print(data)
#         return data

#         # response_items = []
#         # response_items_decoded = []
#         # for i in range(0, (message_length + 1) * 4, 4):
#         #     word = response_body[i:(i+4)]
#         #     item = int.from_bytes(word)
#         #     response_items.append(item)
#         #     item_decoded = struct.unpack('>f', word) if item>1000000 else None
#         #     response_items_decoded.append(item_decoded)
#         #     if verbosity>1:
#         #         # print(f"response item{(i-2)/4:3.0f}: {item} {struct.unpack('>f', response_body[i:(i+4)])} {response_body[i:(i+4)]}")
#         #         print(f"item {i/4:3.0f} (bytes {i:4.0f}-{i+3:4.0f}): {item} {item_decoded} {word}")


# __acoem_decode_logged_data(response=response, verbosity=verbosity)